<a href="https://colab.research.google.com/github/skfkeh/Crawling_Naver/blob/master/json_%ED%8C%8C%EC%9D%BC_%EB%82%98%EB%88%84%EA%B8%B0_CSV%ED%8C%8C%EC%9D%BC_%EA%B0%90%EC%8B%B8%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os
import re

# --- 설정 ---
INPUT_FILENAME = 'chat_log.json'
OUTPUT_DIR = 'split_logs'  # 결과를 저장할 폴더 이름

def sanitize_filename(name):
    """파일명으로 사용할 수 없는 문자를 밑줄(_)로 변경합니다."""
    # Windows 및 다른 OS에서 파일명으로 부적합한 문자 제거
    return re.sub(r'[\\/*?:"<>|]', '_', name)

def split_chat_logs():
    """chat_log.json 파일을 읽어 방별로 분리하여 저장합니다."""

    # 1. 입력 파일 읽기
    try:
        with open("/content/chat_log.json", 'r', encoding='utf-8') as f:
            all_logs = json.load(f)
        print(f"✅ '{INPUT_FILENAME}' 파일을 성공적으로 읽었습니다. (총 {len(all_logs)}개 메시지)")
    except FileNotFoundError:
        print(f"❌ 오류: '{INPUT_FILENAME}' 파일을 찾을 수 없습니다.")
        print("스크립트와 같은 폴더에 파일이 있는지 확인해주세요.")
        return
    except json.JSONDecodeError:
        print(f"❌ 오류: '{INPUT_FILENAME}' 파일이 올바른 JSON 형식이 아닙니다.")
        return

    # 2. 방 이름(room)을 기준으로 데이터 그룹화
    grouped_logs = {}
    for log in all_logs:
        room_name = log.get('room')
        if room_name:
            if room_name not in grouped_logs:
                grouped_logs[room_name] = []
            grouped_logs[room_name].append(log)

    print(f"\n총 {len(grouped_logs)}개의 채팅방을 발견했습니다. 파일 생성을 시작합니다...")

    # 3. 결과를 저장할 폴더 생성
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # 4. 그룹화된 데이터를 별도의 파일로 저장
    for room_name, messages in grouped_logs.items():
        # 파일명으로 사용하기 안전한 이름으로 변경
        safe_room_name = sanitize_filename(room_name)
        output_filename = f"{safe_room_name}_chat_log.json"
        output_path = os.path.join(OUTPUT_DIR, output_filename)

        with open(output_path, 'w', encoding='utf-8') as f:
            # ensure_ascii=False: 한글이 깨지지 않게 저장
            # indent=2: JSON 파일을 사람이 보기 좋게 정렬
            json.dump(messages, f, ensure_ascii=False, indent=2)

        print(f"  - '{output_path}' 파일 생성 완료 ({len(messages)}개 메시지)")

    print(f"\n🎉 모든 작업이 완료되었습니다! '{OUTPUT_DIR}' 폴더를 확인해주세요.")


if __name__ == "__main__":
    split_chat_logs()

✅ 'chat_log.json' 파일을 성공적으로 읽었습니다. (총 150개 메시지)

총 5개의 채팅방을 발견했습니다. 파일 생성을 시작합니다...
  - 'split_logs/DEBUG ROOM_chat_log.json' 파일 생성 완료 (12개 메시지)
  - 'split_logs/정승환_chat_log.json' 파일 생성 완료 (14개 메시지)
  - 'split_logs/친구v_chat_log.json' 파일 생성 완료 (55개 메시지)
  - 'split_logs/정승환, 최훈홍, 최정아, 정지윤, 양회창_chat_log.json' 파일 생성 완료 (57개 메시지)
  - 'split_logs/람대장_chat_log.json' 파일 생성 완료 (12개 메시지)

🎉 모든 작업이 완료되었습니다! 'split_logs' 폴더를 확인해주세요.


In [ ]:
import pandas as pd

df = pd.read_csv('sqld_quiz.csv', header=None)
df.to_csv('sqld_quiz_quoted.csv', index=False, header=False, quoting=1)

In [ ]:
import csv

# 입력 & 출력 파일명
input_path = "sqld_quiz.csv"
output_path = "sqld_quiz_quoted.csv"

# 큰따옴표로 모든 필드를 감싸는 CSV 저장
with open(input_path, 'r', encoding='utf-8') as infile, open(output_path, 'w', encoding='utf-8', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
    for row in reader:
        writer.writerow(row)

In [ ]:
import csv
import sqlite3
from collections import defaultdict

# --- 설정 ---
CSV_FILENAME = 'sqld_quiz.csv'
DB_FILENAME = 'sqld_quiz.db' # 생성될 DB 파일 이름

def migrate_csv_to_sqlite():
    """CSV 파일을 읽어 SQLite 데이터베이스로 마이그레이션합니다."""

    # 1. CSV 파일 읽고 문제 번호별로 데이터 그룹화
    grouped_quizzes = defaultdict(lambda: {'question': '', 'context': '', 'answer': '', 'options': []})

    try:
        with open(CSV_FILENAME, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            for row in reader:
                if len(row) < 6:
                    continue

                q_id, q_text, q_context, q_answer, opt_num, opt_text = row

                # 정답이 여러 개인 문제는 스킵 (예: '1|2')
                if '|' in q_answer or ',' in q_answer:
                    continue

                grouped_quizzes[q_id]['question'] = q_text
                grouped_quizzes[q_id]['context'] = q_context
                grouped_quizzes[q_id]['answer'] = q_answer
                grouped_quizzes[q_id]['options'].append({'num': opt_num, 'text': opt_text})

        print(f"✅ '{CSV_FILENAME}' 파일에서 {len(grouped_quizzes)}개의 문제를 성공적으로 읽었습니다.")

    except FileNotFoundError:
        print(f"❌ 오류: '{CSV_FILENAME}' 파일을 찾을 수 없습니다.")
        return

    # 2. SQLite 데이터베이스 연결 및 테이블 생성
    conn = sqlite3.connect(DB_FILENAME)
    cur = conn.cursor()

    cur.execute('''
        CREATE TABLE IF NOT EXISTS Questions (
            id INTEGER PRIMARY KEY,
            question_text TEXT NOT NULL,
            context_text TEXT,
            correct_option INTEGER NOT NULL
        )
    ''')

    cur.execute('''
        CREATE TABLE IF NOT EXISTS Options (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            question_id INTEGER NOT NULL,
            option_number INTEGER NOT NULL,
            option_text TEXT NOT NULL,
            FOREIGN KEY (question_id) REFERENCES Questions (id)
        )
    ''')
    print("✅ 데이터베이스 테이블을 준비했습니다.")

    # 3. 그룹화된 데이터를 DB에 삽입
    for q_id, data in grouped_quizzes.items():
        try:
            # Questions 테이블에 삽입
            cur.execute(
                "INSERT INTO Questions (id, question_text, context_text, correct_option) VALUES (?, ?, ?, ?)",
                (int(q_id), data['question'], data['context'], int(data['answer']))
            )
            # Options 테이블에 삽입
            for option in data['options']:
                cur.execute(
                    "INSERT INTO Options (question_id, option_number, option_text) VALUES (?, ?, ?)",
                    (int(q_id), int(option['num']), option['text'])
                )
        except ValueError:
            print(f"⚠️ 경고: 문제 번호 '{q_id}' 또는 정답 '{data['answer']}'가 숫자가 아니어서 건너뜁니다.")
            continue

    conn.commit()
    conn.close()

    print(f"🎉 모든 작업 완료! '{DB_FILENAME}' 파일이 생성되었습니다.")
    print("이 파일을 스마트폰의 'sdcard/msgbot/database/quiz/' 폴더로 옮겨주세요.")

if __name__ == "__main__":
    migrate_csv_to_sqlite()

✅ 'sqld_quiz.csv' 파일에서 493개의 문제를 성공적으로 읽었습니다.
✅ 데이터베이스 테이블을 준비했습니다.
🎉 모든 작업 완료! 'sqld_quiz.db' 파일이 생성되었습니다.
이 파일을 스마트폰의 'sdcard/msgbot/database/quiz/' 폴더로 옮겨주세요.


In [ ]:
import csv
import os
from collections import defaultdict
import math

def split_sqld_quiz_csv(input_filename="sqld_quiz.csv", num_files=10):
    """
    SQLD 퀴즈 CSV 파일을 문제 단위로 묶어 여러 개의 파일로 분할합니다.

    Args:
        input_filename (str): 분할할 원본 CSV 파일 이름.
        num_files (int): 분할할 파일의 개수.
    """
    # 1. 입력 파일이 존재하는지 확인
    if not os.path.exists(input_filename):
        print(f"오류: 입력 파일 '{input_filename}'을 찾을 수 없습니다.")
        return

    print(f"'{input_filename}' 파일을 읽는 중입니다...")

    # 2. 문제 번호를 기준으로 모든 행을 그룹화
    # defaultdict를 사용하여 키가 없을 때 빈 리스트를 자동으로 생성합니다.
    # 이렇게 하면 문제(선택지 포함)가 흩어지지 않고 함께 처리됩니다.
    questions = defaultdict(list)
    try:
        # utf-8 인코딩으로 파일을 엽니다 (한글 포함).
        with open(input_filename, mode='r', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            for row in reader:
                # 비어 있는 행은 건너뜁니다.
                if not row:
                    continue
                # 첫 번째 컬럼(문제번호)을 키로 사용하여 행들을 그룹화합니다.
                question_id = row[0]
                questions[question_id].append(row)
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
        return

    # 3. 분할할 청크(파일당 문제 개수) 계산
    # 전체 문제 목록을 고정된 순서로 만듭니다.
    all_question_ids = list(questions.keys())
    total_questions = len(all_question_ids)

    if total_questions == 0:
        print("파일에 데이터가 없습니다. 작업을 중단합니다.")
        return

    # 각 파일에 들어갈 문제의 개수를 계산합니다 (올림 처리).
    chunk_size = math.ceil(total_questions / num_files)
    print(f"총 {total_questions}개의 고유 문제를 찾았습니다. 각 파일에 약 {chunk_size}개의 문제를 할당합니다.")

    # 4. 그룹화된 데이터를 여러 파일에 나누어 쓰기
    file_counter = 1
    for i in range(0, total_questions, chunk_size):
        # 현재 파일에 쓸 문제 ID들을 슬라이싱합니다.
        current_question_ids_chunk = all_question_ids[i:i + chunk_size]

        output_filename = f"sqld_quiz_{file_counter}.csv"

        try:
            # newline='' 옵션은 csv 모듈 사용 시 불필요한 빈 줄이 생기는 것을 방지합니다.
            with open(output_filename, mode='w', newline='', encoding='utf-8') as outfile:
                # quoting=csv.QUOTE_ALL: 모든 필드를 큰따옴표로 묶도록 설정합니다.
                # 이렇게 하면 각 행의 모든 컬럼이 따옴표로 감싸져서 출력됩니다.
                writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)

                # 현재 청크에 해당하는 문제들의 모든 행을 파일에 씁니다.
                for q_id in current_question_ids_chunk:
                    # writerows는 리스트의 각 요소를 한 행씩 씁니다.
                    writer.writerows(questions[q_id])

            print(f"'{output_filename}' 파일이 성공적으로 생성되었습니다.")
            file_counter += 1

        except Exception as e:
            print(f"'{output_filename}' 파일을 쓰는 중 오류가 발생했습니다: {e}")

    print(f"\n총 {file_counter - 1}개의 파일로 분할이 완료되었습니다.")


if __name__ == "__main__":
    split_sqld_quiz_csv()


'sqld_quiz.csv' 파일을 읽는 중입니다...
총 503개의 고유 문제를 찾았습니다. 각 파일에 약 51개의 문제를 할당합니다.
'sqld_quiz_1.csv' 파일이 성공적으로 생성되었습니다.
'sqld_quiz_2.csv' 파일이 성공적으로 생성되었습니다.
'sqld_quiz_3.csv' 파일이 성공적으로 생성되었습니다.
'sqld_quiz_4.csv' 파일이 성공적으로 생성되었습니다.
'sqld_quiz_5.csv' 파일이 성공적으로 생성되었습니다.
'sqld_quiz_6.csv' 파일이 성공적으로 생성되었습니다.
'sqld_quiz_7.csv' 파일이 성공적으로 생성되었습니다.
'sqld_quiz_8.csv' 파일이 성공적으로 생성되었습니다.
'sqld_quiz_9.csv' 파일이 성공적으로 생성되었습니다.
'sqld_quiz_10.csv' 파일이 성공적으로 생성되었습니다.

총 10개의 파일로 분할이 완료되었습니다.
